In [1]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,AveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [14]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from FS.pso import jfs   # change this to switch algorithm 
import matplotlib.pyplot as plt

In [2]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)

In [3]:
train_generator=train_data_gen.flow_from_directory(

   'data/train',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.


In [4]:
validation_generator=validation_data_gen.flow_from_directory(

   'data/test',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 7178 images belonging to 7 classes.


In [16]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))


emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001,decay=1e-6),metrics=['accuracy'])

In [10]:
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/10
448/448 [==============================] - 209s 466ms/step - loss: 1.8045 - accuracy: 0.2608 - val_loss: 1.7130 - val_accuracy: 0.3410
Epoch 2/10
448/448 [==============================] - 207s 461ms/step - loss: 1.6345 - accuracy: 0.3625 - val_loss: 1.5554 - val_accuracy: 0.4043
Epoch 3/10
448/448 [==============================] - 209s 467ms/step - loss: 1.5405 - accuracy: 0.4083 - val_loss: 1.4796 - val_accuracy: 0.4337
Epoch 4/10
448/448 [==============================] - 206s 459ms/step - loss: 1.4743 - accuracy: 0.4341 - val_loss: 1.4276 - val_accuracy: 0.4588
Epoch 5/10
448/448 [==============================] - 208s 465ms/step - loss: 1.4201 - accuracy: 0.4589 - val_loss: 1.3811 - val_accuracy: 0.4734
Epoch 6/10
448/448 [==============================] - 205s 457ms/step - loss: 1.3657 - accuracy: 0.4805 - val_loss: 1.3276 - val_accuracy: 0.4954
Epoch 7/10
448/448 [==============================] - 206s 460ms/step - loss: 1.3213 - accuracy: 0.4995 - val_loss: 1.2971 -

In [11]:
model_json=emotion_model.to_json()
with open("emotion_model.json","w") as json_file:
    json_file.write(model_json)
    
    
    
emotion_model.save_weights('emotion_model.h5')

In [12]:
emotion_model.evaluate(validation_generator)

113/113 [==============================] - 13s 112ms/step - loss: 1.2081 - accuracy: 0.5383


[1.2081116437911987, 0.5383114814758301]